## Initial visualization （generating pictures）

In [1]:
!pip install xlrd

In [7]:
import xlrd
import os
import time
from selenium import webdriver             
import folium

'''census tract'''
file_location= "./CensusTracts.xls"
workbook = xlrd.open_workbook(file_location)
sheet = workbook.sheet_by_index(0)
s=[]
for i in range(sheet.nrows-1):
    s.append(i)
    
k =[]
for i in range(sheet.nrows-1):
    s[i] = sheet.cell_value((i+1),0).split('(')[3]
    s[i] = s[i].split(')')[0]
    k.append([])
    for j in range(len(s[i].split(', '))):
        k[i].append(s[i].split(', ')[j])
'''census tract'''

'''Area information'''
file_location1 = './community_coordinate.xlsx'
workbook1 = xlrd.open_workbook(file_location1)
sheet1 = workbook1.sheet_by_index(0)
for i in range(sheet1.nrows):
    Area = [[0 for x in range(2)] for y in range(78)]
for i in range(sheet1.nrows):
    j = int(sheet1.cell_value((i),0))
    Area[j][0] = sheet1.cell_value((i),1)
    Area[j][1] = sheet1.cell_value((i),2)
'''Area information'''

files=[]
'''Trip_data'''
dir_path="./"
def save_map(m,filename,pngname):
    browser = webdriver.Chrome('./chromedriver')
    print(dir_path + pngname)
    delay = 4
    fn=filename
    tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
    print(tmpurl)
    m.save(fn)
    #use chrome driver, if you have not used it before, download it at https://sites.google.com/a/chromium.org/chromedriver/downloads
    browser.get(tmpurl)
    #Give the map tiles some time to load
    time.sleep(delay)
    #take screenshot
    browser.save_screenshot(dir_path+pngname)
    #browser.quit()
    browser.quit()

file_location2 = "./Day_One2013.xlsx"
workbook2 = xlrd.open_workbook(file_location2)
sheet2 = workbook2.sheet_by_index(0)

time_stamp=[]
start_com=[]
end_com=[]
pace=[]
for row in range(sheet2.nrows-1):
    time_stamp.append(sheet2.cell_value((row+1),1))
    start_com.append(sheet2.cell_value((row+1),6))
    end_com.append(sheet2.cell_value((row+1),3))
    pace.append(sheet2.cell_value((row+1),5))
def count_number(A1, A2, start_row, end_row):
    n = 0
    for i in range(start_row, end_row):
        if start_com[i] == A1 and end_com[i] == A2 or start_com[i] == A2 and end_com[i] == A1:
            n = n+1
    return n
def aver_pace(A1, A2, start_row, end_row):
    pac = 0
    for i in range(start_row, end_row):
        if start_com[i] == A1 and end_com[i] == A2 or start_com[i] == A2 and end_com[i] == A1:
            pac = pace[i] + pac
    if count_number(A1, A2, start_row, end_row) == 0:
        pac = 0
    else:        
        pac = pac/count_number(A1, A2, start_row, end_row)
    return pac
        
Tim = 0
previous_row = 0  
for i in range(sheet2.nrows-2):
    if time_stamp[i+1] != time_stamp[i]:
        Tim = Tim+1
        '''
    if time_stamp[i+1] != time_stamp[i] and Tim-1==25:  
        previous_row = i+1    
    if time_stamp[i+1] != time_stamp[i] and Tim-1>25 and Tim-1<=26:
        '''
    if time_stamp[i+1] != time_stamp[i]:
        m = folium.Map(
                location=[41.923776,-87.775710],
                zoom_start=12,
                tiles='openstreetmap'
                )
        for w in range(sheet.nrows-1):
            list_node = [[0 for x in range(2)] for y in range(len(s[w].split(', ')))]
            for r in range(len(s[w].split(', '))):
                list_node[r][1] = float(k[w][r].split(' ')[0])
                list_node[r][0] = float(k[w][r].split(' ')[1]) 
            census_tract = folium.PolyLine(
                    list_node,
                    weight=1,
                    color = 'blue'
                    )
            m.add_children(census_tract)     
        for j in range(1,78):
            for q in range(j,78):
                if aver_pace(j, q, previous_row, i+1) <= 0.068627:
                    Trip = folium.PolyLine(
                            [Area[j], Area[q]],
                            weight= count_number(j, q, previous_row, i+1)/2,
                            color = 'green'                                                       
                        )
                elif aver_pace(j, q, previous_row, i+1) > 0.068627 and aver_pace(j, q, previous_row, i+1) <= 0.092593:
                     Trip = folium.PolyLine(
                            [Area[j], Area[q]],
                            weight= count_number(j, q, previous_row, i+1)/2,
                            color = 'yellow'                                                       
                        )
                elif aver_pace(j, q, previous_row, i+1) > 0.092593 and aver_pace(j, q, previous_row, i+1) <= 0.133333:
                     Trip = folium.PolyLine(
                            [Area[j], Area[q]],
                            weight= count_number(j, q, previous_row, i+1)/2,
                            color = 'orange'                                                       
                        )
                elif aver_pace(j, q, previous_row, i+1) > 0.133333 and aver_pace(j, q, previous_row, i+1) <= 27.361111:
                     Trip = folium.PolyLine(
                            [Area[j], Area[q]],
                            weight= count_number(j, q, previous_row, i+1)/2,
                            color = 'red'                                                       
                        )
                m.add_children(Trip)
        file_temp=dir_path+'flow_{}.png'.format(Tim-1)
        filename='flow_{}.html'.format(Tim-1)
        pngname='flow_{}.png'.format(Tim-1)
        save_map(m,filename,pngname)    
        files.append(file_temp)
        previous_row = i+1               
'''Trip_data'''

./flow_0.png
file:///Users/gabeshindnes/Documents/MTPsp17/NYCTraffic/flow_0.html
./flow_1.png
file:///Users/gabeshindnes/Documents/MTPsp17/NYCTraffic/flow_1.html
./flow_2.png
file:///Users/gabeshindnes/Documents/MTPsp17/NYCTraffic/flow_2.html
./flow_3.png
file:///Users/gabeshindnes/Documents/MTPsp17/NYCTraffic/flow_3.html
./flow_4.png
file:///Users/gabeshindnes/Documents/MTPsp17/NYCTraffic/flow_4.html
./flow_5.png
file:///Users/gabeshindnes/Documents/MTPsp17/NYCTraffic/flow_5.html
./flow_6.png
file:///Users/gabeshindnes/Documents/MTPsp17/NYCTraffic/flow_6.html
./flow_7.png
file:///Users/gabeshindnes/Documents/MTPsp17/NYCTraffic/flow_7.html
./flow_8.png
file:///Users/gabeshindnes/Documents/MTPsp17/NYCTraffic/flow_8.html
./flow_9.png
file:///Users/gabeshindnes/Documents/MTPsp17/NYCTraffic/flow_9.html
./flow_10.png
file:///Users/gabeshindnes/Documents/MTPsp17/NYCTraffic/flow_10.html
./flow_11.png
file:///Users/gabeshindnes/Documents/MTPsp17/NYCTraffic/flow_11.html
./flow_12.png
file:///Us

KeyboardInterrupt: 

## Resize the images

In [ ]:
dir_path = "./DayOne2013/"
import PIL
from PIL import Image
for i in range(0,95):
    image = Image.open(dir_path+'flow_{}.png'.format(i))
    img = image.resize((2100,1304), PIL.Image.ANTIALIAS)
img.save(dir_path+'flow_test_{}.png'.format(i))

## Draw text in the pictures 

In [ ]:
import xlrd

'''Area information'''
file_location1 = './community_coordinate.xlsx'
workbook1 = xlrd.open_workbook(file_location1)
sheet1 = workbook1.sheet_by_index(0)
for i in range(sheet1.nrows):
    Area = [[0 for x in range(2)] for y in range(78)]
for i in range(sheet1.nrows):
    j = int(sheet1.cell_value((i),0))
    Area[j][0] = sheet1.cell_value((i),1)
    Area[j][1] = sheet1.cell_value((i),2)
'''Area information'''

file_location2 = "./Day_One2013.xlsx"
workbook2 = xlrd.open_workbook(file_location2)
sheet2 = workbook2.sheet_by_index(0)
time_stamp=[]
start_com=[]
end_com=[]
pace=[]
for row in range(sheet2.nrows-1):
    time_stamp.append(sheet2.cell_value((row+1),1))
    start_com.append(sheet2.cell_value((row+1),6))
    end_com.append(sheet2.cell_value((row+1),3))
    pace.append(sheet2.cell_value((row+1),5))
def count_number(A1, A2, start_row, end_row):
    n = 0
    for i in range(start_row, end_row):
        if start_com[i] == A1 and end_com[i] == A2 or start_com[i] == A2 and end_com[i] == A1:
            n = n+1
    return n
def aver_pace(A1, A2, start_row, end_row):
    pac = 0
    for i in range(start_row, end_row):
        if start_com[i] == A1 and end_com[i] == A2 or start_com[i] == A2 and end_com[i] == A1:
            pac = pace[i] + pac
    if count_number(A1, A2, start_row, end_row) == 0:
        pac = 0
    else:        
        pac = pac/count_number(A1, A2, start_row, end_row)
    return pac

dir_path = "./"
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw

def image_add_text(file1,text1,text2,new_file):

    image = Image.open(file1)
    (width1, height1) = image.size
    fnt = ImageFont.truetype(dir_path+"fonts/timeburnernormal.ttf", 50)
    fnt1 = ImageFont.truetype(dir_path+"fonts/Times New Romance.ttf", 40)
    fnt2 = ImageFont.truetype(dir_path+"fonts/Times New Romance.ttf", 40)
    #Area coordinates - picture position
    intial_po = [42.001745,-87.954131] #by observation
    center = [41.923776,-87.77571]
    draw = ImageDraw.Draw(image)
    for j in range(1,78):
        hei = (Area[j][0]-intial_po[0])*(height1/2)/(center[0]-intial_po[0])
        wid = (Area[j][1]-intial_po[1])*(width1/2)/(center[1]-intial_po[1])
        draw.text((wid, hei),"{}".format(j),font=fnt2,fill=(0,0,0,255))
    # Drawing the text on the picture
    draw.text((width1-200, 50),text1,font=fnt,fill=(0,0,0,255))
    draw.text((width1-200, 100),text2,font=fnt,fill=(0,0,0,255))
    draw.text((20,height1/2-140),'Pace (in minutes/mile) between corresponding areas:',font=fnt1,fill=(0,0,0,255))
    draw = ImageDraw.Draw(image)
    # Save the image(overlay the original one) 
    image.save(new_file)
    
def image_add_pace(file1, start_row, end_row, new_file):
    image = Image.open(file1)
    (width1, height1) = image.size
    fnt = ImageFont.truetype(dir_path+"fonts/Times New Romance.ttf", 30)
    draw = ImageDraw.Draw(image)
    n = 0
    for j in range(1,78):
        for q in range(j,78):
            if count_number(j, q, start_row, end_row)!=0:
                draw.text((50+250*int((n*30)/(height1/2)), height1/2-70+(n*30)%int(height1/2)),"{}".format(j)+' to '+"{}".format(q)+':',font=fnt,fill=(0,0,0,255))
                draw.text((170+250*int((n*30)/(height1/2)), height1/2-70+(n*30)%int(height1/2)),"{0:.4f}".format(aver_pace(j, q, start_row, end_row)),font=fnt,fill=(0,0,0,255))
                n = n+1
    # Save the image(overlay the original one) 
    image.save(new_file)
    
final_files=[]
text1 = '00:00'
text2 = '1/1/2013'
Tim = 0
previous_row = 0  
for i in range(sheet2.nrows-2):
    if time_stamp[i+1] != time_stamp[i]:
        Tim = Tim+1
    '''
    if time_stamp[i+1] != time_stamp[i] and Tim-1==125:  
        previous_row = i+1    
    if time_stamp[i+1] != time_stamp[i] and Tim-1>125 and Tim-1<=126:
    '''
    if time_stamp[i+1] != time_stamp[i]:
        time_minute = ((Tim-1)*15)%1440
        if int(time_minute/60) < 10: 
            if time_minute%60 == 0:
                text1='0'+"{}".format(int(time_minute/60))+':'+'0'+"{}".format(time_minute%60)
            else:
                text1='0'+"{}".format(int(time_minute/60))+':'+"{}".format(time_minute%60)
        else:
            if time_minute%60 == 0:
                text1="{}".format(int(time_minute/60))+':'+'0'+"{}".format(time_minute%60)
            else:
                text1="{}".format(int(time_minute/60))+':'+"{}".format(time_minute%60)
        text2='1/'+"{0:.1f}".format(int((Tim-1)*15/1440)+1)[0:1]+'/2013'
        image_add_text(dir_path+'flow_test_{}.png'.format(Tim-1),text1,text2,dir_path+'flow_final_{}.png'.format(Tim-1))       
        image_add_pace(dir_path+'flow_final_{}.png'.format(Tim-1), previous_row, i+1, dir_path+'flow_final_{}.png'.format(Tim-1)) 
        final_files.append(dir_path+'flow_final_{}.png'.format(Tim-1))
previous_row = i+1 

## Make video

In [ ]:
dir_path = "./DayOne2013/"
final_files=[]
for i in range(95):
    final_files.append(dir_path+'flow_final_{}.png'.format(i))
import imageio
images = []
for file in final_files:
    images.append(imageio.imread(file))
imageio.mimsave(dir_path+'Trips_Time series.mp4', images,fps=6)